In [ ]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments, pipeline
from sklearn.metrics import classification_report, roc_auc_score
import torch
from sklearn.model_selection import train_test_split

#Download and Prepare the Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Toxic Comment Classification/train.csv')
df.head()

In [ ]:
df_small = df.sample(n=5000, random_state=42)

In [ ]:
#Extract labels and texts
labels = df_small[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
texts = df_small['comment_text'].tolist()

#Split data (80% train, 20% validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

#Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

#Tokenize texts
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

#Create PyTorch dataset
class ToxicCommentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ToxicCommentDataset(train_encodings, train_labels)
val_dataset = ToxicCommentDataset(val_encodings, val_labels)

In [ ]:
# Initialize DistilBERT for multi-label classification (6 labels)
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=6,
    problem_type="multi_label_classification"
)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [ ]:
def compute_metrics(pred):
    logits, labels = pred
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid for probabilities
    aucs = []
    for i in range(labels.shape[1]):
        try:
            auc = roc_auc_score(labels[:, i], probs[:, i])
        except:
            auc = np.nan
        aucs.append(auc)
    return {
        'roc_auc': np.nanmean(aucs)
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
model.save_pretrained('./toxic_comment_classifier')
tokenizer.save_pretrained('./toxic_comment_classifier')

In [ ]:
classifier = pipeline(
    "text-classification",
    model='./toxic_comment_classifier',
    tokenizer='./toxic_comment_classifier',
    function_to_apply='sigmoid',  # For multi-label
    top_k=6  # Show all 6 labels
)

# Test with a sample comment
sample_comment = "I hate you so much, you worthless piece of trash!"
results = classifier(sample_comment)

label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
result = results[0]
for i, label in enumerate(label_names):
    print(f"{label}: {result[i]['score']:.2f}")